In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler

from imblearn.over_sampling import SMOTE

In [ ]:
try:
    df = pd.read_csv(r'.archive\creditcard.csv')
    print(f"Dataset loaded. Shape: {df.shape}")
    print("Class Distribution:")

    print(df['Class'].value_counts(normalize=True))
except FileNotFoundError:
    print("Error: The file 'creditcard.csv' was not found.")
